In [ ]:
import nltk
nltk.download('punkt')
nltk.download("movie_reviews")
nltk.download('stopwords')
nltk.download('words')
nltk.download('state_union')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('wordnet')
nltk.download('letter-recognition.data')
!pip install tweepy

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package state_union to /root/nltk_data...
[nltk_data]   Unzipping corpora/state_union.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Error loading letter-r

In [ ]:
from nltk import ngrams
from nltk.corpus import stopwords 
import string
 
stopwords_english = stopwords.words('english')
 
# clean words, i.e. remove stopwords and punctuation
def clean_words(words, stopwords_english):
    words_clean = []
    for word in words:
        word = word.lower()
        if word not in stopwords_english and word not in string.punctuation:
            words_clean.append(word)    
    return words_clean 
 
# feature extractor function for unigram
def bag_of_words(words):    
    for i in range ( len(words) ) : 
         tup = (words[i],)
         words[i]= tup
    words_dictionary = dict([word, True] for word in words)    
    return words_dictionary
 
# feature extractor function for ngrams (bigram)
def bag_of_ngrams(words, n=2):
    words_ng = []
    for item in iter(ngrams(words, n)):
        words_ng.append(item)
    words_dictionary = dict([word, True] for word in words_ng)    
    return words_dictionary
 
# feature extractor function for ngrams (bigram)
def bag_of_3grams(words, n=3):
    words_ng = []
    for item in iter(ngrams(words, n)):
        words_ng.append(item)
    words_dictionary = dict([word, True] for word in words_ng)    
    return words_dictionary
 

from nltk.tokenize import word_tokenize
important_words = ['above', 'below', 'off', 'over', 'under', 'more', 'most', 'such', 'no', 'nor', 'not', 'only', 'so', 'than', 'too', 'very', 'just', 'but']
stopwords_english_for_bigrams = set(stopwords_english) - set(important_words)
 
  
  
  
# let's define a new function that extracts all features
# i.e. that extracts all unigram and bigrams and trigrams features
def bag_of_all_words(words, mode):
    words_clean = clean_words(words, stopwords_english)
    words_clean_for_bigrams = clean_words(words, stopwords_english_for_bigrams)
    
    unigram_features = bag_of_words(words_clean)
    bigram_features = bag_of_ngrams(words_clean_for_bigrams)
    trigram_features = bag_of_3grams(words_clean_for_bigrams)
    all_features = unigram_features.copy()
    
    if mode == "unigram":
         return all_features
      
    if mode == "bigram":
         all_features.update(bigram_features)
         return all_features
      
    if mode == "trigram" : 
         all_features.update(trigram_features)
         return all_features
 



In [ ]:
from nltk.corpus import movie_reviews 
 
pos_reviews = []
for fileid in movie_reviews.fileids('pos'):
    words = movie_reviews.words(fileid)
    pos_reviews.append(words)
 
neg_reviews = []
for fileid in movie_reviews.fileids('neg'):
    words = movie_reviews.words(fileid)
    neg_reviews.append(words)
    
    

In [ ]:
# positive reviews feature set
pos_reviews_set = []
for words in pos_reviews:
    pos_reviews_set.append((bag_of_all_words(words, mode = "bigram"), 'pos'))
 

# negative reviews feature set
neg_reviews_set = []
for words in neg_reviews:
    neg_reviews_set.append((bag_of_all_words(words, mode = "bigram"), 'neg'))
    
    
    
    
    
    

In [ ]:
print (len(pos_reviews_set), len(neg_reviews_set)) # Output: (1000, 1000)
 
# radomize pos_reviews_set and neg_reviews_set
# doing so will output different accuracy result everytime we run the program
from random import shuffle 

shuffle(pos_reviews_set)
shuffle(neg_reviews_set)
 
test_set = pos_reviews_set[:200] + neg_reviews_set[:200]
train_set = pos_reviews_set[200:] + neg_reviews_set[200:]
 
print(len(test_set),  len(train_set)) # Output: (400, 1600)

1000 1000
400 1600


In [ ]:
import warnings
warnings.filterwarnings('ignore')
from nltk import classify
from nltk import NaiveBayesClassifier
from nltk.classify.scikitlearn import SklearnClassifier

from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.neighbors import KNeighborsClassifier
from nltk.classify import ClassifierI
from statistics import mode

classifier = NaiveBayesClassifier.train(train_set)
print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, test_set))*100)
classifier.show_most_informative_features(15)


MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(train_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, test_set))*100)

BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(train_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, test_set))*100)

LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(train_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, test_set))*100)

SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(train_set)
print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGDClassifier_classifier, test_set))*100)

##SVC_classifier = SklearnClassifier(SVC())
##SVC_classifier.train(train_set)
##print("SVC_classifier accuracy percent:", (nltk.classify.accuracy(SVC_classifier, test_set))*100)

LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(train_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, test_set))*100)

NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(train_set)
print("NuSVC_classifier accuracy percent:", (nltk.classify.accuracy(NuSVC_classifier, test_set))*100)




Original Naive Bayes Algo accuracy percent: 79.75
Most Informative Features
        ('one', 'worst') = True              neg : pos    =     21.7 : 1.0
          ('uplifting',) = True              pos : neg    =     13.0 : 1.0
       ('quite', 'well') = True              pos : neg    =     11.7 : 1.0
           ('captures',) = True              pos : neg    =     11.0 : 1.0
         ('astounding',) = True              pos : neg    =     11.0 : 1.0
               ('3000',) = True              neg : pos    =     10.3 : 1.0
     ('bad', 'dialogue') = True              neg : pos    =     10.3 : 1.0
             ('regard',) = True              pos : neg    =     10.3 : 1.0
        ('understands',) = True              pos : neg    =      9.7 : 1.0
          ('strongest',) = True              pos : neg    =      9.7 : 1.0
          ('affecting',) = True              pos : neg    =      9.7 : 1.0
         ('makes', 'no') = True              neg : pos    =      9.7 : 1.0
  ('best', 'supporting')

In [ ]:
from nltk.tokenize import word_tokenize
 



def classify_tweet (tweet) :
      tweet_review_tokens = word_tokenize(tweet)
      tweet_review_set = bag_of_all_words(tweet_review_tokens,mode = "bigram")
      neg_count = 0
      pos_count = 0

      prediction = classifier.classify(tweet_review_set)
      if prediction == "pos":
        pos_count +=1
      else:
        neg_count +=1
      print("Niave Bayes classifier predicts as             ",prediction) 


      prediction = MNB_classifier.classify(tweet_review_set)
      if prediction == "pos":
        pos_count +=1
      else:
        neg_count +=1
      print("Multinomial Niave Bayes classifier predicts as ",prediction)

      prediction  = BernoulliNB_classifier.classify(tweet_review_set)
      if prediction == "pos":
        pos_count +=1
      else:
        neg_count +=1
      print("Bernoulli Niave Bayes classifier predicts as   ",prediction)

      prediction = LogisticRegression_classifier.classify(tweet_review_set)
      if prediction == "pos":
        pos_count +=1
      else:
        neg_count +=1
      print("Logistic Regression classifier predicts as     ",prediction)

      prediction = SGDClassifier_classifier.classify(tweet_review_set)
      if prediction == "pos":
        pos_count +=1
      else:
        neg_count +=1
      print("SGD Classifier predicts as                     ",prediction)

      prediction = LinearSVC_classifier.classify(tweet_review_set)
      if prediction == "pos":
        pos_count +=1
      else:
        neg_count +=1
      print("Linear SVC classifier predicts as              ",prediction)

      predition = NuSVC_classifier.classify(tweet_review_set)
      if prediction == "pos":
        pos_count +=1
      else:
        neg_count +=1
      print("Nu SVC classifier predicts as                  ",prediction)



      if pos_count > neg_count:
        print("\nConfidence for the tweet to be positive is " , pos_count/7, "\n\n" )
      else:
        print("\nConfidence for the tweet to be negative is " , neg_count/7, "\n\n")
        
        
        

tweet = " I am happy with how the movie was. The movie was really awesome and the best of all time"
print(tweet, "\n")
classify_tweet (tweet)
tweet = "I really dislike it, I wish it could had been better but had the same old boring and dull story"
print(tweet, "\n")
classify_tweet (tweet)
tweet = "Narendra modi has been one of the most influential and strongest leader in the world. All because of the good deeds that he has done selflessly for the people of India"
print(tweet, "\n")
classify_tweet (tweet)
tweet = "Narendra modi is a failed and a bad prime minister, never impressed me  wih his work. His work shows no growth,he made no efforts and always disappointed me on many grounds"
print(tweet, "\n")
classify_tweet (tweet)

 I am happy with how the movie was. The movie was really awesome and the best of all time 

Niave Bayes classifier predicts as              neg
Multinomial Niave Bayes classifier predicts as  neg
Bernoulli Niave Bayes classifier predicts as    neg
Logistic Regression classifier predicts as      pos
SGD Classifier predicts as                      pos
Linear SVC classifier predicts as               pos
Nu SVC classifier predicts as                   pos

Confidence for the tweet to be positive is  0.5714285714285714 


I really dislike it, I wish it could had been better but had the same old boring and dull story 

Niave Bayes classifier predicts as              neg
Multinomial Niave Bayes classifier predicts as  neg
Bernoulli Niave Bayes classifier predicts as    neg
Logistic Regression classifier predicts as      neg
SGD Classifier predicts as                      neg
Linear SVC classifier predicts as               neg
Nu SVC classifier predicts as                   neg

Confidence for

In [ ]:
from tweepy import Stream
from tweepy import OAuthHandler
import json
from tweepy.streaming import StreamListener

#consumer key, consumer secret, access token, access secret.
ckey="E6TmyKCmlSO2PaIMST3pEdkJF"
csecret="kHNX2TKatjRYweRV5C0HXT1HwgqNyJl2hQTnmZLR7FJl8pVRVL"
atoken="1123356891298422784-4nqHqQbPOESRR5gCzc89Mr5BSaqfu5"
asecret="NhhSW3F48si8gXTacnKCi941MXbbqL3qpKMEASwyQaIJ8"

class listener(StreamListener):
    
    def on_data(self, data):
        all_data = json.loads(data)

        tweet = all_data["text"]
        
        username = all_data["user"]["screen_name"]

        print(username,"\n",tweet,"\n")
        count = + 1
        
        classify_tweet(tweet)
        
        return True

    def on_error(self, status):
        print (status)

auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

twitterStream = Stream(auth, listener())
twitterStream.filter(languages=["en"] , track=["messi"])


aelzodiac 
 RT @GurusDrive: Who is the G•O•A•T? Let's settle this

RT for Lionel Messi

Like for Cristiano Ronaldo

#BARLIV #BarçaLFC https://t.co/HY2k… 

Niave Bayes classifier predicts as              neg
Multinomial Niave Bayes classifier predicts as  neg
Bernoulli Niave Bayes classifier predicts as    neg
Logistic Regression classifier predicts as      neg
SGD Classifier predicts as                      neg
Linear SVC classifier predicts as               neg
Nu SVC classifier predicts as                   neg

Confidence for the tweet to be negative is  1.0 


AyathulaZ 
 RT @ProperFootball2: Messi and Ronaldo both hit the 600 goal mark in the same week. What a legacy these two will leave behind. https://t.co… 

Niave Bayes classifier predicts as              pos
Multinomial Niave Bayes classifier predicts as  pos
Bernoulli Niave Bayes classifier predicts as    neg
Logistic Regression classifier predicts as      neg
SGD Classifier predicts as                      pos
Linear SVC cla

KeyboardInterrupt: ignored

In [ ]:
 import tweepy
  
